Calculate the Call/Put price of a European Option, using the Binomial Model

In [5]:
# importing necessary packages
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
import statistics as stat
import math as m

In [6]:
def binomial_tree(S0, n, u, d):
    tree = [[S0]]  # Stock price at time 0
    for i in range(1, n+1):  # Repeat process equal to the number of specified steps n
        prev_level = tree[-1]  # Previous level is the most recent list in the tree
        new_level = [prev_level[0] * u]  # First element of the new level is the first element of the previous level * u
        for j in range(1, i+1):  # Loop through to calculate the rest of the prices at this level
            new_price = prev_level[j-1] * d  # Next element of the new level is the corresponding element in the previous level * d
            new_level.append(new_price)  # Append the new price to the new level
        tree.append(new_level)  # Add the new level to the tree
    return tree  # Return the full binomial tree


def backwards_induction(terminals, r, dt, u, d, n):
    p = (m.exp(r*dt)-d)/(u-d)  # Risk-neutral probability based on u, d, and the interest rate r
    mytree = [terminals]  # Initialize the tree with the terminal payoff values
    for j in range(1, n+1):  # Work backwards through the tree, one level at a time
        discounted_level = []  # Initialize the discounted level for this step
        for i, item in enumerate(mytree[-1]):  # Loop through the previous (terminal) level
            if i+1 < len(mytree[-1]):  # Ensure we are within bounds of the previous level
                # Calculate the discounted value using the risk-neutral probability
                discounted = m.exp(-r*dt) * (p*mytree[-1][i] + (1-p)*mytree[-1][i+1])
                discounted_level.append(discounted)  # Append discounted value to current level
        mytree.append(discounted_level)  # Append the newly calculated level to the tree
    return mytree  # Return the tree with all levels populated from the terminal to the root


def euro_call(S0, K, r, T, n, u, d):
    dt = T/n  # Calculate the time step for each interval
    BT = binomial_tree(S0, n, u, d)  # Build the binomial stock price tree
    terminal_payoff = []  # Initialize the list to store terminal payoffs for call option
    for i in BT[-1]:  # Loop through the terminal nodes (last level of the tree)
        if i > K:  # If stock price is greater than strike price, calculate the payoff
            terminal_payoff.append(i - K)  # Payoff for call option is (S-K)
        else:
            terminal_payoff.append(0)  # If stock price <= strike price, payoff is 0
    # Perform backwards induction to calculate the option price
    final_tree = backwards_induction(terminal_payoff, r, dt, u, d, n)
    return final_tree[-1][0]  # Return the option price (root of the tree)


def euro_put(S0, K, r, T, n, u, d):
    dt = T/n  # Calculate the time step for each interval
    BT = binomial_tree(S0, n, u, d)  # Build the binomial stock price tree
    terminal_payoff = []  # Initialize the list to store terminal payoffs for put option
    for i in BT[-1]:  # Loop through the terminal nodes (last level of the tree)
        if K > i:  # If strike price is greater than stock price, calculate the payoff
            terminal_payoff.append(K - i)  # Payoff for put option is (K-S)
        else:
            terminal_payoff.append(0)  # If strike price <= stock price, payoff is 0
    # Perform backwards induction to calculate the option price
    final_tree = backwards_induction(terminal_payoff, r, dt, u, d, n)
    return final_tree[-1][0]  # Return the option price (root of the tree)



In [8]:
EC = euro_call(29, 28, 0.05, 0.25, 1000, m.exp(0.15*np.sqrt(0.25/1000)), m.exp(-0.15*np.sqrt(0.25/1000)))
print("European Call Option Price: £", EC)

EP = euro_put(9, 10, 0.04, 2, 2, 1.1, 0.9)
print("European Put Option Price: £", EP)

European Call Option Price: £ 1.686371377305191
European Put Option Price: £ 0.6384107785689576
